In [67]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import sqlite3
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [68]:
basics_ratings = pd.read_csv('../Data/movie_basics_rating.csv')
tmdb = pd.read_csv('../Data/tmdb_df.csv')
budget = pd.read_csv('../Data/movie_budget.csv')
info = pd.read_csv('../Data/movie_info.csv')
gross = pd.read_csv('../Data/bom_movie_gross.csv')

In [69]:
basics_ratings

,Unnamed: 0,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119
...,...,...,...,...,...,...,...,...,...
73851,73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6
73852,73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136
73853,73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8
73854,73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,NaN,6.6,5


In [70]:
tmdb

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
25492,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
25493,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
25494,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
25495,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [71]:
type(tmdb['genre_ids'])

pandas.core.series.Series

In [72]:
genre_map = {28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy',
             80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 
             14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 
             9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 
             10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}
#tmdb['genres'] = tmdb['genre_ids'].apply(lambda x: [genre_map[int(genre_id)] for genre_id in eval(x)])
tmdb['genre_labels'] = tmdb['genre_ids'].apply(lambda x: ', '.join([genre_map[int(genre_id)] for genre_id in eval(x)]))
tmdb

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,genre_labels
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,"Adventure, Fantasy, Family"
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,"Fantasy, Adventure, Animation, Family"
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,"Adventure, Action, Science Fiction"
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,"Animation, Comedy, Family"
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,"Action, Science Fiction, Adventure"
...,...,...,...,...,...,...,...,...,...,...,...
25492,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1,"Horror, Drama"
25493,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1,"Drama, Thriller"
25494,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,"Fantasy, Action, Adventure"
25495,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1,"Family, Adventure, Action"


In [73]:
budget

,Unnamed: 0,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_profit
0,0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09
1,1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08
2,2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-2.002376e+08
3,3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09
4,4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,9.997217e+08
...,...,...,...,...,...,...,...,...
5777,5777,78,"Dec 31, 2018",Red 11,7000.0,0.0,0.000000e+00,-7.000000e+03
5778,5778,79,"Apr 2, 1999",Following,6000.0,48482.0,2.404950e+05,2.344950e+05
5779,5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000.0,1338.0,1.338000e+03,-3.662000e+03
5780,5780,81,"Sep 29, 2015",A Plague So Pleasant,1400.0,0.0,0.000000e+00,-1.400000e+03


In [74]:
info

,Unnamed: 0,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,runtime,studio
0,0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",104 minutes,NaN
1,1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",108 minutes,Entertainment One
2,2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",116 minutes,NaN
3,3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",128 minutes,NaN
4,4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1555,1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",106 minutes,New Line Cinema
1556,1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",88 minutes,Paramount Vantage
1557,1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",111 minutes,NaN
1558,1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",101 minutes,NaN


In [75]:
gross

,Unnamed: 0,title,studio,domestic_gross,foreign_gross,year
0,0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,3,Inception,WB,292600000.0,535700000.0,2010
4,4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010
...,...,...,...,...,...,...
3382,3382,The Quake,Magn.,6200.0,NaN,2018
3383,3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,3384,El Pacto,Sony,2500.0,NaN,2018
3385,3385,The Swan,Synergetic,2400.0,NaN,2018


- merge basics_ratings & tmdb on original_title = basics_ratings_tmdb
- merge budget & gross on movie & title, respectively = budget_gross
- basics_ratings_tmdb & budget_gross on...
- info = irrelevant?

In [76]:
#merging basics_ratings & tmdb
basics_ratings_tmdb = pd.merge(basics_ratings, tmdb, on='original_title', how='inner')
basics_ratings_tmdb

,Unnamed: 0_x,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,Unnamed: 0_y,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count,genre_labels
0,2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,24185,"[35, 18]",299782,en,9.800,2018-11-02,The Other Side of the Wind,7.0,64,"Comedy, Drama"
1,5,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller",4.1,32,5872,"[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26,"Horror, Science Fiction, Adventure"
2,46328,tt4503112,Bigfoot,Bigfoot,2018,NaN,"Action,Horror",4.6,9,5872,"[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26,"Horror, Science Fiction, Adventure"
3,72791,tt9181914,Bigfoot,Bigfoot,2018,86.0,"Animation,Family",2.5,55,5872,"[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26,"Horror, Science Fiction, Adventure"
4,16,tt0249516,Foodfight!,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,8248,8456,"[16, 28, 35, 10751]",116977,en,4.705,2013-05-07,Foodfight!,2.1,46,"Animation, Action, Comedy, Family"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16923,73431,tt9598566,Ave Maria,Ave Maria,2018,74.0,Drama,7.3,30,16520,[53],444182,en,0.600,2015-08-19,Ave Maria,10.0,1,Thriller
16924,73433,tt9600932,The Brink,The Brink,2019,91.0,Documentary,6.2,107,16711,"[14, 18]",361995,en,0.600,2015-08-07,The Brink,8.0,1,"Fantasy, Drama"
16925,73521,tt9647790,Celia,Celia,2018,100.0,Documentary,8.2,8,6338,[18],334204,en,1.400,2012-10-26,Celia,5.7,3,Drama
16926,73560,tt9668594,Belonging,Aidiyet,2019,73.0,"Crime,Drama,Romance",6.3,38,20434,[18],383260,tr,0.600,2016-03-30,Belonging,5.0,1,Drama


In [77]:
basics_ratings_tmdb.drop(['Unnamed: 0_x'], axis=1)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,Unnamed: 0_y,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count,genre_labels
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,24185,"[35, 18]",299782,en,9.800,2018-11-02,The Other Side of the Wind,7.0,64,"Comedy, Drama"
1,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller",4.1,32,5872,"[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26,"Horror, Science Fiction, Adventure"
2,tt4503112,Bigfoot,Bigfoot,2018,NaN,"Action,Horror",4.6,9,5872,"[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26,"Horror, Science Fiction, Adventure"
3,tt9181914,Bigfoot,Bigfoot,2018,86.0,"Animation,Family",2.5,55,5872,"[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26,"Horror, Science Fiction, Adventure"
4,tt0249516,Foodfight!,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,8248,8456,"[16, 28, 35, 10751]",116977,en,4.705,2013-05-07,Foodfight!,2.1,46,"Animation, Action, Comedy, Family"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16923,tt9598566,Ave Maria,Ave Maria,2018,74.0,Drama,7.3,30,16520,[53],444182,en,0.600,2015-08-19,Ave Maria,10.0,1,Thriller
16924,tt9600932,The Brink,The Brink,2019,91.0,Documentary,6.2,107,16711,"[14, 18]",361995,en,0.600,2015-08-07,The Brink,8.0,1,"Fantasy, Drama"
16925,tt9647790,Celia,Celia,2018,100.0,Documentary,8.2,8,6338,[18],334204,en,1.400,2012-10-26,Celia,5.7,3,Drama
16926,tt9668594,Belonging,Aidiyet,2019,73.0,"Crime,Drama,Romance",6.3,38,20434,[18],383260,tr,0.600,2016-03-30,Belonging,5.0,1,Drama


In [78]:
budget_gross = pd.merge(budget,
                       gross,
                       left_on='movie',
                       right_on='title',
                       how='inner')
budget_gross

,Unnamed: 0_x,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,net_profit,Unnamed: 0_y,title,studio,domestic_gross_y,foreign_gross,year
0,1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,330,Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,804600000.0,2011
1,3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09,1875,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015
2,6,7,"Apr 27, 2018",Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,1.748134e+09,3079,Avengers: Infinity War,BV,678800000.0,136900000.0,2018
3,8,9,"Nov 17, 2017",Justice League,300000000.0,229024295.0,6.559452e+08,3.559452e+08,2771,Justice League,WB,229000000.0,428900000.0,2017
4,9,10,"Nov 6, 2015",Spectre,300000000.0,200074175.0,8.796209e+08,5.796209e+08,1877,Spectre,Sony,200100000.0,680600000.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,5667,68,"Apr 27, 2012",Sound of My Voice,135000.0,408015.0,4.294480e+05,2.944480e+05,1019,Sound of My Voice,FoxS,408000.0,NaN,2012
1243,5672,73,"Jun 15, 2012",Your Sister's Sister,120000.0,1597486.0,3.090593e+06,2.970593e+06,971,Your Sister's Sister,IFC,1600000.0,NaN,2012
1244,5679,80,"Jul 10, 2015",The Gallows,100000.0,22764410.0,4.165647e+07,4.155647e+07,1986,The Gallows,WB (NL),22800000.0,20200000.0,2015
1245,5685,86,"Jul 7, 2017",A Ghost Story,100000.0,1594798.0,2.769782e+06,2.669782e+06,2967,A Ghost Story,A24,1600000.0,NaN,2017


In [79]:
df = pd.merge(basics_ratings_tmdb, 
                  budget_gross,
                  left_on='primary_title',
                  right_on='movie',
                  how='inner')
df

,Unnamed: 0_x_x,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,Unnamed: 0_y_x,...,production_budget,domestic_gross_x,worldwide_gross,net_profit,Unnamed: 0_y_y,title_y,studio,domestic_gross_y,foreign_gross,year
0,41,tt0337692,On the Road,On the Road,2012,124.0,"Adventure,Drama,Romance",6.1,37886,5350,...,25000000.0,720828.0,9313302.0,-15686698.0,904,On the Road,IFC,744000.0,8000000.0,2012
1,45101,tt4339118,On the Road,On the Road,2014,89.0,Drama,6.0,6,5350,...,25000000.0,720828.0,9313302.0,-15686698.0,904,On the Road,IFC,744000.0,8000000.0,2012
2,55547,tt5647250,On the Road,On the Road,2016,121.0,Drama,5.7,127,5350,...,25000000.0,720828.0,9313302.0,-15686698.0,904,On the Road,IFC,744000.0,8000000.0,2012
3,47,tt0359950,The Secret Life of Walter Mitty,The Secret Life of Walter Mitty,2013,114.0,"Adventure,Comedy,Drama",7.3,275300,7998,...,91000000.0,58236838.0,187861183.0,96861183.0,1169,The Secret Life of Walter Mitty,Fox,58200000.0,129900000.0,2013
4,51,tt0365907,A Walk Among the Tombstones,A Walk Among the Tombstones,2014,114.0,"Action,Crime,Drama",6.5,105116,11053,...,28000000.0,26017685.0,62108587.0,34108587.0,1577,A Walk Among the Tombstones,Uni.,26300000.0,26900000.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,66324,tt7334528,Uncle Drew,Uncle Drew,2018,103.0,"Comedy,Sport",5.7,9739,24120,...,18000000.0,42469946.0,46527161.0,28527161.0,3175,Uncle Drew,LG/S,42500000.0,4200000.0,2018
1491,66395,tt7349662,BlacKkKlansman,BlacKkKlansman,2018,135.0,"Biography,Crime,Drama",7.5,149005,23864,...,15000000.0,49275340.0,93017335.0,78017335.0,3144,BlacKkKlansman,Focus,49300000.0,44000000.0,2018
1492,66584,tt7388562,"Paul, Apostle of Christ","Paul, Apostle of Christ",2018,108.0,"Adventure,Biography,Drama",6.7,5662,24076,...,5000000.0,17547999.0,25529498.0,20529498.0,3199,"Paul, Apostle of Christ",Affirm,17600000.0,5500000.0,2018
1493,66659,tt7401588,Instant Family,Instant Family,2018,118.0,"Comedy,Drama",7.4,46728,23881,...,48000000.0,67363237.0,119736188.0,71736188.0,3131,Instant Family,Par.,67400000.0,53200000.0,2018


In [80]:
df.isna().sum()

Unnamed: 0_x_x         0
movie_id               0
primary_title          0
original_title         0
start_year             0
runtime_minutes       33
genres                 6
averagerating          0
numvotes               0
Unnamed: 0_y_x         0
genre_ids              0
id_x                   0
original_language      0
popularity             0
release_date_x         0
title_x                0
vote_average           0
vote_count             0
genre_labels           0
Unnamed: 0_x_y         0
id_y                   0
release_date_y         0
movie                  0
production_budget      0
domestic_gross_x       0
worldwide_gross        0
net_profit             0
Unnamed: 0_y_y         0
title_y                0
studio                 0
domestic_gross_y       0
foreign_gross        226
year                   0
dtype: int64

In [82]:
df_profit_genres=df.loc(axis=1)['primary_title', 'genres', 'net_profit']
df_profit_genres
# relevant columns to analyse genres & profits

,primary_title,genres,net_profit
0,On the Road,"Adventure,Drama,Romance",-15686698.0
1,On the Road,Drama,-15686698.0
2,On the Road,Drama,-15686698.0
3,The Secret Life of Walter Mitty,"Adventure,Comedy,Drama",96861183.0
4,A Walk Among the Tombstones,"Action,Crime,Drama",34108587.0
...,...,...,...
1490,Uncle Drew,"Comedy,Sport",28527161.0
1491,BlacKkKlansman,"Biography,Crime,Drama",78017335.0
1492,"Paul, Apostle of Christ","Adventure,Biography,Drama",20529498.0
1493,Instant Family,"Comedy,Drama",71736188.0


In [83]:
df_profit_genres.loc[df_profit_genres.duplicated()]

,primary_title,genres,net_profit
2,On the Road,Drama,-15686698.0
22,Wonder Woman,"Action,Adventure,Fantasy",671133378.0
24,Wonder Woman,Sci-Fi,671133378.0
26,Wonder Woman,"Documentary,Drama,Sport",671133378.0
39,Unstoppable,"Action,Thriller",70720921.0
...,...,...,...
1426,The Square,"Comedy,Drama",-1323738.0
1428,Mandy,"Drama,Thriller",-4572344.0
1430,Mandy,"Action,Fantasy,Horror",-4572344.0
1436,Get Out,"Horror,Mystery,Thriller",250367951.0


In [84]:
df_profit_genres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1495 entries, 0 to 1494
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   primary_title  1495 non-null   object 
 1   genres         1489 non-null   object 
 2   net_profit     1495 non-null   float64
dtypes: float64(1), object(2)
memory usage: 46.7+ KB


In [85]:
df_profit_genres.isna().sum()

primary_title    0
genres           6
net_profit       0
dtype: int64

In [86]:
df_profit_genres.duplicated()

0       False
1       False
2        True
3       False
4       False
        ...  
1490    False
1491    False
1492    False
1493    False
1494    False
Length: 1495, dtype: bool

In [87]:
df_profit_genres.drop_duplicates()

,primary_title,genres,net_profit
0,On the Road,"Adventure,Drama,Romance",-1.568670e+07
1,On the Road,Drama,-1.568670e+07
3,The Secret Life of Walter Mitty,"Adventure,Comedy,Drama",9.686118e+07
4,A Walk Among the Tombstones,"Action,Crime,Drama",3.410859e+07
5,Jurassic World,"Action,Adventure,Sci-Fi",1.433855e+09
...,...,...,...
1490,Uncle Drew,"Comedy,Sport",2.852716e+07
1491,BlacKkKlansman,"Biography,Crime,Drama",7.801734e+07
1492,"Paul, Apostle of Christ","Adventure,Biography,Drama",2.052950e+07
1493,Instant Family,"Comedy,Drama",7.173619e+07


In [89]:
df_profit_genres.sort_values('net_profit', ascending=False).head(20)

,primary_title,genres,net_profit
1367,Avengers: Infinity War,"Action,Adventure,Sci-Fi",1.748134e+09
5,Jurassic World,"Action,Adventure,Sci-Fi",1.433855e+09
845,Black Panther,"Action,Adventure,Sci-Fi",1.148258e+09
844,Black Panther,"Action,Adventure,Sci-Fi",1.148258e+09
1420,Jurassic World: Fallen Kingdom,"Action,Adventure,Sci-Fi",1.135773e+09
359,Frozen,"Adventure,Drama,Sport",1.122470e+09
362,Frozen,"Adventure,Animation,Comedy",1.122470e+09
361,Frozen,"Adventure,Animation,Comedy",1.122470e+09
360,Frozen,"Adventure,Drama,Sport",1.122470e+09
1078,Minions,"Adventure,Animation,Comedy",1.086336e+09


In [90]:
df_profit_genres.loc[df_profit_genres.duplicated()]

,primary_title,genres,net_profit
2,On the Road,Drama,-15686698.0
22,Wonder Woman,"Action,Adventure,Fantasy",671133378.0
24,Wonder Woman,Sci-Fi,671133378.0
26,Wonder Woman,"Documentary,Drama,Sport",671133378.0
39,Unstoppable,"Action,Thriller",70720921.0
...,...,...,...
1426,The Square,"Comedy,Drama",-1323738.0
1428,Mandy,"Drama,Thriller",-4572344.0
1430,Mandy,"Action,Fantasy,Horror",-4572344.0
1436,Get Out,"Horror,Mystery,Thriller",250367951.0


In [91]:
df_profit_genres=df_profit_genres.drop_duplicates('primary_title')
df_profit_genres

,primary_title,genres,net_profit
0,On the Road,"Adventure,Drama,Romance",-1.568670e+07
3,The Secret Life of Walter Mitty,"Adventure,Comedy,Drama",9.686118e+07
4,A Walk Among the Tombstones,"Action,Crime,Drama",3.410859e+07
5,Jurassic World,"Action,Adventure,Sci-Fi",1.433855e+09
6,The Rum Diary,"Comedy,Drama",-2.345527e+07
...,...,...,...
1490,Uncle Drew,"Comedy,Sport",2.852716e+07
1491,BlacKkKlansman,"Biography,Crime,Drama",7.801734e+07
1492,"Paul, Apostle of Christ","Adventure,Biography,Drama",2.052950e+07
1493,Instant Family,"Comedy,Drama",7.173619e+07


In [93]:
df_profit_genres.sort_values('net_profit', ascending=False).head(20)

,primary_title,genres,net_profit
1367,Avengers: Infinity War,"Action,Adventure,Sci-Fi",1.748134e+09
5,Jurassic World,"Action,Adventure,Sci-Fi",1.433855e+09
844,Black Panther,"Action,Adventure,Sci-Fi",1.148258e+09
1420,Jurassic World: Fallen Kingdom,"Action,Adventure,Sci-Fi",1.135773e+09
359,Frozen,"Adventure,Drama,Sport",1.122470e+09
1078,Minions,"Adventure,Animation,Comedy",1.086336e+09
1119,Avengers: Age of Ultron,"Action,Adventure,Sci-Fi",1.072414e+09
1314,Incredibles 2,"Action,Adventure,Animation",1.042521e+09
503,Aquaman,"Action,Adventure,Fantasy",9.868946e+08
1394,The Fate of the Furious,"Action,Crime,Thriller",9.848463e+08


In [95]:
df_profit_genres.loc[df_profit_genres.duplicated()]

,primary_title,genres,net_profit


In [96]:
df_profit_genres.shape

(1133, 3)

In [97]:
type(df_profit_genres.genres)

pandas.core.series.Series

In [100]:
df_profit_genres.assign(genres=df_profit_genres.genres.str.split(",")).explode('genres')

,primary_title,genres,net_profit
0,On the Road,Adventure,-15686698.0
0,On the Road,Drama,-15686698.0
0,On the Road,Romance,-15686698.0
3,The Secret Life of Walter Mitty,Adventure,96861183.0
3,The Secret Life of Walter Mitty,Comedy,96861183.0
...,...,...,...
1493,Instant Family,Comedy,71736188.0
1493,Instant Family,Drama,71736188.0
1494,Hereditary,Drama,60133905.0
1494,Hereditary,Horror,60133905.0
